In [1]:
import numpy as np
import random
import time
from typing import Tuple, Optional, Dict

In [2]:
def make_problem(seed: int = 42) -> Dict[str, np.ndarray]:
    rng = np.random.default_rng(seed=seed)
    NUM_KNAPSACKS = 100
    NUM_ITEMS = 5000
    NUM_DIMENSIONS = 100

    values = rng.integers(0, 1000, size=NUM_ITEMS, dtype=np.int64)
    weights = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS), dtype=np.int64)
    constraints = rng.integers(
        10000,
        1000 * 2 * NUM_ITEMS // NUM_KNAPSACKS,
        size=(NUM_KNAPSACKS, NUM_DIMENSIONS),
        dtype=np.int64
    )

    return {
        "num_knapsacks": NUM_KNAPSACKS,
        "num_items": NUM_ITEMS,
        "num_dims": NUM_DIMENSIONS,
        "values": values,
        "weights": weights,
        "constraints": constraints
    }

In [3]:
def evaluate_full(sol: np.ndarray,
                  values: np.ndarray,
                  weights: np.ndarray,
                  constraints: np.ndarray) -> Tuple[int, np.ndarray, bool, int]:
    num_knapsacks, num_dims = constraints.shape
    loads = np.zeros((num_knapsacks, num_dims), dtype=np.int64)
    for i, k in enumerate(sol):
        if k > 0:
            loads[k - 1] += weights[i]
    total_value = int(values[sol > 0].sum())
    violation_matrix = np.maximum(0, loads - constraints)
    violation = int(violation_matrix.sum())
    feasible = (violation == 0)
    return total_value, loads, feasible, violation

In [4]:
def greedy_initial(values: np.ndarray,
                   weights: np.ndarray,
                   constraints: np.ndarray) -> np.ndarray:
    n = values.shape[0]
    num_knapsacks, num_dims = constraints.shape
    denom = 1.0 + weights.sum(axis=1).astype(np.float64)
    ratio = values.astype(np.float64) / denom
    order = np.argsort(-ratio)

    sol = np.zeros(n, dtype=np.int32)
    loads = np.zeros((num_knapsacks, num_dims), dtype=np.int64)

    for i in order:
        wi = weights[i]
        for k in range(num_knapsacks):
            if np.all(loads[k] + wi <= constraints[k]):
                sol[i] = k + 1
                loads[k] += wi
                break
    return sol

In [5]:
def compute_loads_and_value(sol: np.ndarray,
                            values: np.ndarray,
                            weights: np.ndarray,
                            num_knapsacks: int,
                            num_dims: int) -> Tuple[np.ndarray, int]:
    loads = np.zeros((num_knapsacks, num_dims), dtype=np.int64)
    total_value = 0
    for i, k in enumerate(sol):
        if k > 0:
            loads[k - 1] += weights[i]
            total_value += int(values[i])
    return loads, total_value

In [6]:
def delta_move(i: int, old_k: int, new_k: int,
               weights: np.ndarray, values: np.ndarray,
               loads: np.ndarray, constraints: np.ndarray) -> Tuple[int, int]:
    w = weights[i]
    if old_k == 0 and new_k != 0:
        dv = int(values[i])
    elif old_k != 0 and new_k == 0:
        dv = -int(values[i])
    else:
        dv = 0

    delta_violation = 0
    if old_k != 0:
        kidx = old_k - 1
        cur = loads[kidx]
        new = cur - w
        cap = constraints[kidx]
        delta_violation += int(np.maximum(0, new - cap).sum() - np.maximum(0, cur - cap).sum())

    if new_k != 0:
        kidx = new_k - 1
        cur = loads[kidx]
        new = cur + w
        cap = constraints[kidx]
        delta_violation += int(np.maximum(0, new - cap).sum() - np.maximum(0, cur - cap).sum())

    return dv, delta_violation


In [7]:
def local_search_incremental(sol: np.ndarray,
                             values: np.ndarray,
                             weights: np.ndarray,
                             constraints: np.ndarray,
                             loads: np.ndarray,
                             total_value: int,
                             penalty_lambda: float,
                             max_evals: int,
                             neighbor_sample_size: int,
                             time_budget_s: Optional[float],
                             start_time: float) -> Tuple[np.ndarray, np.ndarray, int]:

    n = sol.shape[0]
    num_knapsacks = constraints.shape[0]
    evals = 0
    cur_violation = int(np.maximum(0, loads - constraints).sum())
    cur_score = total_value - penalty_lambda * cur_violation

    while evals < max_evals:
        if time_budget_s is not None and (time.time() - start_time) > time_budget_s:
            break

        improved = False
        sample_size = min(neighbor_sample_size, n)

        # Shuffle items to ensure every item is considered
        indices = np.arange(n)
        np.random.shuffle(indices)

        for idx in indices[:sample_size]:
            i = idx
            old_k = int(sol[i])
            new_k = random.randrange(0, num_knapsacks + 1)
            if new_k == old_k:
                continue

            dv, dviol = delta_move(i, old_k, new_k, weights, values, loads, constraints)
            new_score = (total_value + dv) - penalty_lambda * (cur_violation + dviol)
            evals += 1

            # Accept move if it improves score OR reduces violation
            if new_score > cur_score or dviol < 0:
                if old_k != 0:
                    loads[old_k - 1] -= weights[i]
                if new_k != 0:
                    loads[new_k - 1] += weights[i]
                sol[i] = new_k
                total_value += dv
                cur_violation += dviol
                cur_score = total_value - penalty_lambda * cur_violation
                improved = True
                break

        # Continue until evals exhausted
        if not improved:
            break

    return sol, loads, total_value


In [8]:
def perturb(sol: np.ndarray,
            loads: np.ndarray,
            weights: np.ndarray,
            values: np.ndarray,
            constraints: np.ndarray,
            strength_fraction: float) -> Tuple[np.ndarray, np.ndarray, int]:

    n = sol.shape[0]
    num_knapsacks = constraints.shape[0]
    num_changes = max(1, int(n * strength_fraction))
    indices = random.sample(range(n), num_changes)

    for i in indices:
        old_k = int(sol[i])
        if old_k != 0:
            loads[old_k - 1] -= weights[i]
        # Ensure new_k != old_k for actual change
        new_k = old_k
        while new_k == old_k:
            new_k = random.randrange(0, num_knapsacks + 1)
        sol[i] = new_k
        if new_k != 0:
            loads[new_k - 1] += weights[i]

    total_value = int(values[sol > 0].sum())
    return sol, loads, total_value


In [9]:
def iterated_local_search(values: np.ndarray,
                          weights: np.ndarray,
                          constraints: np.ndarray,
                          seed: int = 42,
                          max_outer_iters: int = 200,
                          local_search_budget: int = 50000,
                          neighbor_sample_size: int = 1000,
                          perturb_strength: float = 0.05,
                          penalty_lambda: float = 100.0,
                          time_limit_s: Optional[float] = 300.0,
                          verbose: bool = True) -> Dict:

    random.seed(seed)
    np.random.seed(seed)
    start_time = time.time()

    num_knapsacks = constraints.shape[0]
    n = values.shape[0]

    sol = greedy_initial(values, weights, constraints)
    loads, total_value = compute_loads_and_value(sol, values, weights, num_knapsacks, constraints.shape[1])

    sol, loads, total_value = local_search_incremental(
        sol, values, weights, constraints, loads, total_value,
        penalty_lambda=penalty_lambda,
        max_evals=local_search_budget,
        neighbor_sample_size=neighbor_sample_size,
        time_budget_s=time_limit_s,
        start_time=start_time
    )

    best_feasible_val = -1
    best_feasible_sol = None

    # Initial check
    tv, _, feas, _ = evaluate_full(sol, values, weights, constraints)
    if feas:
        best_feasible_val = int(tv)
        best_feasible_sol = sol.copy()

    cur_score = total_value - penalty_lambda * int(np.maximum(0, loads - constraints).sum())

    for outer in range(1, max_outer_iters + 1):
        if time_limit_s is not None and (time.time() - start_time) > time_limit_s:
            if verbose:
                print("Time limit reached; terminating outer loop.")
            break

        cand_sol = sol.copy()
        cand_loads = loads.copy()
        cand_sol, cand_loads, cand_total_value = perturb(
            cand_sol, cand_loads, weights, values, constraints,
            strength_fraction=perturb_strength
        )

        cand_sol, cand_loads, cand_total_value = local_search_incremental(
            cand_sol, values, weights, constraints, cand_loads, cand_total_value,
            penalty_lambda=penalty_lambda,
            max_evals=local_search_budget,
            neighbor_sample_size=neighbor_sample_size,
            time_budget_s=time_limit_s,
            start_time=start_time
        )

        cand_violation = int(np.maximum(0, cand_loads - constraints).sum())
        cand_score = cand_total_value - penalty_lambda * cand_violation

        # Update best feasible immediately if candidate is feasible
        if cand_violation == 0 and (best_feasible_val == -1 or cand_total_value > best_feasible_val):
            best_feasible_val = int(cand_total_value)
            best_feasible_sol = cand_sol.copy()

        # Acceptance for exploration
        accept = cand_score > cur_score or random.random() < 0.005
        if accept:
            sol = cand_sol
            loads = cand_loads
            total_value = cand_total_value
            cur_score = cand_score

        if verbose and (outer % max(1, max_outer_iters // 20) == 0 or outer <= 5):
            elapsed = time.time() - start_time
            print(f"Outer {outer}/{max_outer_iters} | Best feasible: {best_feasible_val} | "
                  f"CurScore: {cur_score} | Time: {elapsed:.1f}s")

    total_time = time.time() - start_time
    return {
        "best_feasible_value": best_feasible_val,
        "best_feasible_sol": best_feasible_sol,
        "best_overall_score": int(cur_score),
        "best_overall_sol": sol,
        "time_s": total_time
    }


In [10]:
problem = make_problem(seed=42)
values = problem["values"]
weights = problem["weights"]
constraints = problem["constraints"]

result = iterated_local_search(
    values=values,
    weights=weights,
    constraints=constraints,
    seed=42,
    max_outer_iters=200,
    local_search_budget=50000,
    neighbor_sample_size=1000,
    perturb_strength=0.01,
    penalty_lambda=100.0,
    time_limit_s=300.0,
    verbose=True
)

print("\n=== ILS Summary ===")
print("Best feasible value:", result["best_feasible_value"])
print("Total time (s):", result["time_s"])
if result["best_feasible_sol"] is not None:
    tv, loads, feas, viol = evaluate_full(result["best_feasible_sol"], values, weights, constraints)
    print("Final feasible check -> feasible:", feas, "value:", tv, "violation:", viol)
else:
    print("No feasible solution found.")

Outer 1/200 | Best feasible: 1818128 | CurScore: 1818128.0 | Time: 1.0s
Outer 2/200 | Best feasible: 1818128 | CurScore: 1818128.0 | Time: 1.2s
Outer 3/200 | Best feasible: 1818128 | CurScore: 1818128.0 | Time: 1.3s
Outer 4/200 | Best feasible: 1818128 | CurScore: 1818128.0 | Time: 1.5s
Outer 5/200 | Best feasible: 1818128 | CurScore: 1818128.0 | Time: 1.7s
Outer 10/200 | Best feasible: 1818128 | CurScore: 1818128.0 | Time: 2.5s
Outer 20/200 | Best feasible: 1818128 | CurScore: 1818346.0 | Time: 4.3s
Outer 30/200 | Best feasible: 1818128 | CurScore: 1818346.0 | Time: 6.1s
Outer 40/200 | Best feasible: 1818128 | CurScore: 1818346.0 | Time: 8.2s
Outer 50/200 | Best feasible: 1818128 | CurScore: 1818346.0 | Time: 10.0s
Outer 60/200 | Best feasible: 1818128 | CurScore: 1818346.0 | Time: 11.9s
Outer 70/200 | Best feasible: 1818128 | CurScore: 1827879.0 | Time: 14.2s
Outer 80/200 | Best feasible: 1818128 | CurScore: 1790046.0 | Time: 16.3s
Outer 90/200 | Best feasible: 1818128 | CurScore: 17